# IBM APPLIED DATA SCIENCE CAPSTONE - THE BATTLE OF NEIGHBORHOODS

## TABLE OF CONTENT
+ **THE BUSINESS CASE**
+ **THE DATA**
+ **THE METHODOLOGY**

## THE BUSINESS CASE 
My fiance and I love pizza. We eat it minimum once a week and we are pretty picky about it. The best pizza place in our neighbourhood is called [L'Osteria](https://losteria.net/en/). Pizzas are huge, drinks are decent, atmosphere is great and prices are reasonable - perfect place for young people like us!
<br>
<br>
You probably now wonder what this has to do with the Capstone Project for the [IBM - Data Science Professional Certficiate](https://www.coursera.org/professional-certificates/ibm-data-science). L'Osteria is a restaurant chain for italian food, founded in 1999 in Germany. The chain currently has 139 restaurants in Europe, of which the majority is located in Germany. As the chain is looking to expand further, it continuously needs to scout new locations for their next restaurants. As that can be an extremely difficult and long process, this Capstone Project aims to build an analysis tool that recommends neighbourhoods which could be a good fit for new L'Osteria locations. In result, the process of scouting new neihbourhoods should be more time and cost efficient.
<br>
<br>
The tool is going to be based on a k-means clustering algorithm that uses data from Foursquare, Facebook and governmental websites. As my ressources for this project are limited, the tool will be focused on Germany only. However, the approach could be adapted to any other country or region, if enough ressources are provided. Further ressources would also enable adding more socio-economic data which would make the algorithm more precise. 

## THE DATA 
As already mentioned above, we are going to use data from different sources. In this section, I will provide you with a detailed description for each data source used and provide you with the code you need to download/upload the data.
<br>
<br>
### FOURSQUARE DATA
[Foursquare](https://foursquare.com/) is an app that lets its users rank all different kinds of venues all over the world. By doing so,Foursquare has build one of the most comprehensive geospatial data sets in the world. The data set covers an extremely large amount of venue and user data that allows you to access everything from a venue's name, location or menu up to user ratings, comments or pictures of each venue. And the best part: Foursquare provides developers an [API](https://developer.foursquare.com/) that is (in limits) free to use! 
<br>
<br>
Even though Foursquare provides an incredible variety of data, we will use the venue categories (e.g. sushi restaurant, park, sports club, italian restaurant, etc.) for our analysis only. While it would most likely increase the success of our algorithm to include more information such as rankings or types of user data, we will not include this data due to ressource limitations.
<br>
<br>
To being able to access the Foursquare data during our analysis, you must [sign up](https://developer.foursquare.com/) for a developer account with Foursquare first. Afterwards you need to log into your account and navigate to "My Apps". Here, you can create yourself your own client ID and secret. Save these credentials as txt files and name them according to the code below.
<br>
Once you have created your credentials, you need to create an access token. For that, just follow [this guide](https://developer.foursquare.com/docs/places-api/authentication/) and save the access token in a txt file with the name provided below.

In [1]:
# Import working library
import pandas as pd

# Set credentials
client_id = list(pd.read_csv('FOURSQUARE_CLIENT_ID.txt').columns)[0]
client_secret = list(pd.read_csv('FOURSQUARE_CLIENT_SECRET.txt').columns)[0]
access_token = list(pd.read_csv('FOURSQUARE_ACCESS_TOKEN.txt').columns)[0]

# Check out
print("Foursquare credentials loaded.")

Foursquare credentials loaded.


You are now set to use the Foursquare API! We will use these credentials at a later point. For now, we continue with our next data source.

### FACEBOOK POPULATION DATA
Of all the bad Corona did to us, it also had a good side effect: The big data miners of the world opened up and shared some of their digital gold with the rest of us. So did Facebook by publishing large datasets based on their app usage. You can find a detailed description of their "Data for Good" project under this [link](https://dataforgood.fb.com/). Most of the project's data is publicly accessible via [The Humanitarian Data Exchange](https://data.humdata.org/organization/facebook) or [AWS](https://registry.opendata.aws/dataforgood-fb-hrsl/).
<br>
<br>
One of the datasets provided by Facebook contains population density information. According to Facebook, it is the world's most accurate population dataset. It is divided by countries, each containing seven datasets for the distribution of various populations in relation to their respective coordinates: 
<br>
<br>
1. Overall population density 
<br>
2. Women
<br>
3. Men
<br> 
4. Children (ages 0-5) 
<br>
5. Youth (ages 15-24) 
<br>
6. Elderly (ages 60+) 
<br>
7. Women of reproductive age (ages 15-49)
<br>
<br>
Due to limited computing ressources, we will focus on the German datasets for women and men only. You can download them under the following links: 
<br>
<br>
1. [DEU_men.csv.zip](https://data.humdata.org/dataset/7d08e2b0-b43b-43fd-a6a6-a308f222cdb2/resource/9b14d65e-ebe1-4509-8680-1f21becc75d9/download/deu_men_2019-08-03_csv.zip)
<br>
2. [DEU_women.csv.zip](https://data.humdata.org/dataset/7d08e2b0-b43b-43fd-a6a6-a308f222cdb2/resource/26b8ec5b-e40b-40af-a4fc-fcd85bfb0818/download/deu_women_2019-08-03_csv.zip)
<br>
<br>
Once downloaded, we can execute the code below. It will load the data, transform and standardize it. Furthermore, we are merging the two datasets to receive one comprehensive population dataset. Finally, we will add a total population column to our dataset which is the sum of all male and female population. 

In [2]:
# Load density data
male_population = pd.read_csv('DEU_men_2019-08-03.csv')
female_population = pd.read_csv('DEU_women_2019-08-03.csv')

# Standardize columns
male_population.rename(columns={"latitude":"LATITUDE", "longitude":"LONGITUDE", "population":"MALE_POPULATION"}, inplace=True)
female_population.rename(columns={"latitude":"LATITUDE", "longitude":"LONGITUDE", "population":"FEMALE_POPULATION"}, inplace=True)

# Merge on geolocation
population = male_population.merge(female_population, how='outer', on=['LATITUDE', 'LONGITUDE'])

# Calculate total population column
population['TOTAL_POPULATION'] = population['MALE_POPULATION'] + population['FEMALE_POPULATION']

# Visualize
print(population.shape)
population.head(10)

(32433580, 5)


,LATITUDE,LONGITUDE,MALE_POPULATION,FEMALE_POPULATION,TOTAL_POPULATION
0,48.814306,8.459306,1.451906,1.552587,3.004493
1,48.815694,8.480972,1.451906,1.552587,3.004493
2,48.804861,8.445139,1.451906,1.552587,3.004493
3,48.787361,8.449306,1.451906,1.552587,3.004493
4,48.819028,8.464306,1.451906,1.552587,3.004493
5,48.824583,8.415139,1.451906,1.552587,3.004493
6,48.775417,8.455972,1.451906,1.552587,3.004493
7,48.817639,8.482917,1.451906,1.552587,3.004493
8,48.769306,8.432917,1.451906,1.552587,3.004493
9,48.817361,8.485417,1.451906,1.552587,3.004493


### GERMAN BORDER DATA
As our Facebook data compromises over 32 million unique geospatial data points that are randomized, there is no way for us to filter this data according to cities or neighborhoods in Germany. One solution to this problem would be to reverse engineer the coordinates with libraries like geopy. However, the accessible APIs via geopy only allow a limited number of calls per day. For +32 million datapoints it would require month do download all necessary data this way. Of course there is always the option to upgrade to a paid enterprise plan to speed up this process. However, this would require financial ressources that I currently do not have. 
<br>
<br>
Luckily, there is another option! The German government publishes shape files of all municipalities Germany. You might ask yourself now how that is going to help us in regards to neighborhoods. The shapefiles provide us with unique polygons for each municipality that we can use to filter our Facebook data. Once filtered, the amount of coordinates that need to be reverse engineered is going to be much less time and cost intensive.
<br>
<br>
The shape files can be downloaded on this [website](https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/verwaltungsgebiete.html) for free. For this excercise, I have used the [vg250_12-31.utm32s.shape.ebenen](https://daten.gdz.bkg.bund.de/produkte/vg/vg250_ebenen_1231/aktuell/vg250_12-31.utm32s.shape.ebenen.zip) dataset. Once you click the link it will direct you to download the respective zip file. Alternatively, you can access the needed files on the Github repository. 
<br>
<br>
The following code will allow you to load the shape file and convert it into a pandas data frame:

In [3]:
# Import working libraries
import pandas as pd

# Define function to read shape files
def read_shapefile(shp_path):
	"""
	Read a shapefile into a Pandas dataframe with a 'coords' column holding
	the geometry information. This uses the pyshp package.
	"""
    # Import working libraries
	import shapefile

	# Read file, parse out the records and shapes
	sf = shapefile.Reader(shp_path)
	fields = [x[0] for x in sf.fields][1:]
	records = sf.records()
	shps = [s.points for s in sf.shapes()]

	# Write into a dataframe
	df = pd.DataFrame(columns=fields, data=records)
	df = df.assign(coords=shps)

    # Check out
	return df

# Set path
path = r"C:\Users\maurice.buettgenbach\OneDrive - Aquila Capital Management GmbH\Desktop\Desktop\Private\IBM\10_Capstone project\IBM_The_battle_of_neighborhoods\VG250_GEM.shp"

# Run function
municipalities = read_shapefile(path)

#Visualize
print(municipalities.shape)
municipalities.head()

(11139, 27)


,ADE,GF,BSG,ARS,AGS,SDV_ARS,GEN,BEZ,IBZ,BEM,...,FK_S3,NUTS,ARS_0,AGS_0,WSK,DEBKG_ID,RS,SDV_RS,RS_0,coords
0,6,4,1,010010000000,01001000,010010000000,Flensburg,Stadt,60,kreisfrei,...,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000E5MA,010010000000,010010000000,010010000000,"[(526513.7529476011, 6075133.41194521), (52654..."
1,6,4,1,010020000000,01002000,010020000000,Kiel,Stadt,60,kreisfrei,...,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000004J,010020000000,010020000000,010020000000,"[(575841.569459631, 6032148.031753651), (57586..."
2,6,4,1,010030000000,01003000,010030000000,Lübeck,Stadt,60,kreisfrei,...,R,DEF03,010030000000,01003000,2006-02-01,DEBKGDL20000DYMA,010030000000,010030000000,010030000000,"[(623056.1506336611, 5983746.445214357), (6231..."
3,6,4,1,010040000000,01004000,010040000000,Neumünster,Stadt,60,kreisfrei,...,R,DEF04,010040000000,01004000,1970-04-26,DEBKGDL20000E4SA,010040000000,010040000000,010040000000,"[(565015.6516448742, 6000637.513467715), (5651..."
4,6,4,1,010510011011,01051011,010510011011,Brunsbüttel,Stadt,61,--,...,R,DEF05,010510011011,01051011,2009-01-01,DEBKGDL20000E2IK,010510011011,010510011011,010510011011,"[(510789.9284805175, 5977425.101718364), (5109..."


As we can see, the dataframe covers a lot of information that we don't need. So let us drop these columns and have a closer look on the columns that matter:

In [4]:
# Kick out unnecessary columns
municipalities = municipalities[[
	"GEN",
	"BEZ",
	"coords"
]]

# Visualize
municipalities.head()

,GEN,BEZ,coords
0,Flensburg,Stadt,"[(526513.7529476011, 6075133.41194521), (52654..."
1,Kiel,Stadt,"[(575841.569459631, 6032148.031753651), (57586..."
2,Lübeck,Stadt,"[(623056.1506336611, 5983746.445214357), (6231..."
3,Neumünster,Stadt,"[(565015.6516448742, 6000637.513467715), (5651..."
4,Brunsbüttel,Stadt,"[(510789.9284805175, 5977425.101718364), (5109..."


Much better! So what are we looking at? The column "GEN" shows the respective municipality and colume "BEZ" its type (e.g. "Stadt" which is German for "City").
<br>
<br>
The column "coords" contains the border information in form of a polygon. Having a closer look at the data, we can see that the coordinates are not in standard WGS84 (GPS) format. A quick research shows that the German authorities (for whatever reason) still work with the ETRS89 format. Since the libraries we will use in later steps all work with WGS84, we need to convert the coords column first. 
<br>
<br>
Luckily, we can use the [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html) library for this. Pyproj is a complex library that allows you to convert coordinates from (nearly) every kind of system to another. To use the library correctly, one needs to know the so called "[EPSG code](https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset)" for the respective coordinate system. In our case, ETRS89 has the code "25832" and WGS84 has "44326". The following code will now convert the coordinates to GPS standard:

In [5]:
# Import workin library
from tqdm import tqdm # Progress visualization
from pyproj import Transformer # Coordinate transformation

# Define coordinate systems
system_in = "EPSG:25832" # ETRS89
system_out = "EPSG:4326" # WGS84

# Define transformer
transformer = Transformer.from_crs(system_in, system_out)

# Loop through each entry in dataset
for i in tqdm(range(len(municipalities.index))):
    # Loop through each coordinate pair in polygon
    for index, tuple in enumerate(municipalities['coords'][i]):
        # Get coordinates in ETRS89 format
        lat_in = tuple[0] 
        lng_in = tuple[1]
        # Execute transformer
        transformed_coordinates = transformer.transform(lat_in, lng_in)
        # Parse transformed coordinates back to dataset
        municipalities['coords'][i][index] = transformed_coordinates

# Spacer
print()
print()

# Visualize
municipalities.head()

100%|██████████| 11139/11139 [00:24<00:00, 451.74it/s]




,GEN,BEZ,coords
0,Flensburg,Stadt,"[(54.82264090832692, 9.412664108896106), (54.8..."
1,Kiel,Stadt,"[(54.43137670797368, 10.169158185796851), (54...."
2,Lübeck,Stadt,"[(53.98736518615582, 10.876835000180577), (53...."
3,Neumünster,Stadt,"[(54.14971926107453, 9.995446122384308), (54.1..."
4,Brunsbüttel,Stadt,"[(53.94509189359393, 9.164391038995467), (53.9..."


### OPEN STREET MAP
To reverse geocode our filtered coordinates, we will use the publicly available geospatial data by [OpenStreetMap](https://wiki.openstreetmap.org/wiki/Main_Page). OpenStreetMap is a non profit organization that collects geospatial data and makes in available to the public. We will use this data to reverse geocode our filtered coordinates. We can do so by using the service [Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim) via the [GeoPy](https://geopy.readthedocs.io/en/stable/) library. More details to this in the library section. 

### L'OSTERIA DATA
Finally, we will use the geospatial data from the L'Osteria website to localize their existing restaurants. As the chain is rather big and it would take too long to extract the coordinates from the website by hand, we will use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to scrape the address data from the website. We then can use Nominatim to receive their coordinates at a later stage.
<br>
<br>
Usually, once would use BeautifulSoup to scrape data directly from a website. However, in this case, the website has an event button that hides most of the restaurants. We could use another library called [Selenium](https://www.selenium.dev/) to click the button for us automatically - but that would be pretty much overkill as we can also click on it 10 times manually and save it in our working directory. Since this is not an ongoing analysis but a one time thing, an automation definitely makes no sense. 
<br>
<br>
The following code scrapes the data we need from the downloaded html file (available in the repository) and transforms it so that we can use it later:

In [6]:
# Import working libraries
import requests # Request handling
from bs4 import BeautifulSoup # HTML search
import re # String handling
import ast # String handling

# Open document and transform to BS object
with open("Restaurants - L'Osteria.htm") as page:
    soup = BeautifulSoup(page, 'html.parser')

# Define results
results = soup.find(id='losteria-restaurants-list-wrapper')

# Find elements
page_elements = results.find_all('div', class_='address')

# Create variable to store data in
restaurants = pd.DataFrame()

# Iterate through found elements
for page_element in tqdm(page_elements):
    # Isolate address
    element = str(page_element)
    # Get rid of unnecessary characters
    element = element.replace("\n", "")
    element = element.replace("\r", "")
    # Get rid of spaces
    element = element.replace(" ", "")
    # Search for address in between <div>
    search_result = re.search(">(.*)<", element)
    # Store search result in new variable
    address = search_result.group(1)
    # Get street
    street = address.split(",")[0]
    # Add space between street name and house number
    street = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", street).strip()
    # Add space before Capital letter
    street = re.sub(r"(?<=\w)([A-Z])", r" \1", street).strip()
    # Get ZIP code and city
    address = address.split(",")[1]
    # Isolate ZIP code
    zip_code = re.split('(\d+)', address)[1]
    # Isolate city
    city = re.split('(\d+)', address)[2]
    # Create temporary df
    temp_df = pd.DataFrame()
    # Add data to df
    temp_df['STREET'] = [street]
    temp_df['ZIP_CODE'] = zip_code
    temp_df['CITY'] = city
    # Append to variable
    restaurants = restaurants.append(temp_df)
# Reset index
restaurants.reset_index(drop=True, inplace=True)

# Replace cryptic html for German characters
restaurants = restaurants.replace('Ã¤', "ae", regex=True) # For "ä"
restaurants = restaurants.replace('Ã¼', "ue", regex=True) # For "ü"
restaurants = restaurants.replace('Ã¶', "oe", regex=True) # For "ö"
restaurants = restaurants.replace('ÃŸ', "ss", regex=True) # For "ß"

# Add spacer for output
print()
print()

# Visualize variable
print("df shape:", restaurants.shape)
restaurants.head(10)

100%|██████████| 121/121 [00:00<00:00, 446.04it/s]

df shape: (121, 3)



,STREET,ZIP_CODE,CITY
0,Gut-Daemme-Strasse 1,52070,Aachen
1,Franziskanergasse 5,92224,Amberg
2,Pearl-S.-Buck-Strasse 12,86156,Augsburg
3,Albert-Schenavsky-Str. 2,86165,Augsburg
4,Maximilianstrasse 83,95444,Bayreuth
5,Hilda-Geiringer-Weg 4,10557,Berlin
6,Alt Mahlsdorf 88,12623,Berlin
7,Memhardstrasse 3,10178,Berlin
8,Mildred-Harnack-Str. 11,10243,Berlin
9,Savignyplatz 5,10623,Berlin


## THE METHODOLOGY
Alright! We have our base datasets ready. In this section, we will explore our data, apply further standardization/transformation where needed and finally use some machine learning techniques. 

### ANALYZING L'OSTERIA LOCATIONS
Let us begin with our L'Osteria data. We have successfully scraped the locations of each restaurant shown on their website. As we are going to limit our analysis to Germany, we will need to check whether all of these addresses actually lie within Germany. And what is the easiest way to do that? Right, let's look on a map!
<br>
<br>
But before we can do that, we unfortunately need to get the coordinates for each address. We can do so by using the service of Nominatim that we can access via the [Geopy](https://geopy.readthedocs.io/en/stable/) library. The following code takes our existing addresses and fetches their coordinates from the Nominatim API:

In [7]:
# Import working library
from geopy.geocoders import Nominatim

# Create new variable
lats = []
longs = []

# Set geolocator and set user agent
geolocator = Nominatim(user_agent="german_italian_restaurant_analysis")

# Get temporary df
temp_df = restaurants.copy()

# Loop through df and get addresses for coordinates
for i in tqdm(range(len(temp_df))):
    # Get address
    address = temp_df.iloc[i, 0:1].values[0]
    # Get ZIP code
    zip_code = temp_df.iloc[i, 1:2].values[0]
    # Get city
    city = temp_df.iloc[i, 2:3].values[0]
    # Define full address
    full_address = "{Address}, {ZIPCode}, {City}".format(Address=address, ZIPCode=zip_code, City=city)
    # Get full location details
    location = geolocator.geocode(full_address)
    # Check if location is empty
    if location is not None:
        # Get latitude
        latitude = location.latitude
        # Append latitude
        lats.append(latitude)
    # If empty, add 0
    else:
        # Ste lats to 0
        latitude = 0
        # Append latitude
        lats.append(latitude)
    # Check if location is empty
    if location is not None:
        # Get longitude
        longitude = location.longitude
        # Append longitude
        longs.append(longitude)
    # If empty, add 0
    else:
        longitude = 0
        # Append finding to df
        longs.append(longitude)


# Add longs and lats to restaurant df
restaurants['LATITUDE'] = lats
restaurants['LONGITUDE'] = longs

# Visualize
restaurants.head()

100%|██████████| 121/121 [01:00<00:00,  2.00it/s]


,STREET,ZIP_CODE,CITY,LATITUDE,LONGITUDE
0,Gut-Daemme-Strasse 1,52070,Aachen,50.797335,6.107450
1,Franziskanergasse 5,92224,Amberg,49.446234,11.855468
2,Pearl-S.-Buck-Strasse 12,86156,Augsburg,48.371837,10.865182
3,Albert-Schenavsky-Str. 2,86165,Augsburg,48.381857,10.931547
4,Maximilianstrasse 83,95444,Bayreuth,49.944196,11.571159


As we can see, downloading all information took us ~1 minute. That is not bad. However, Nominatim sometimes cannot find the addresses provided. For these cases, we have added a simple if loop in the code above. Our loop adds "0" to latitude and longitude when Nominatim cannot find the address. So let us check how many addresses are affected:

In [8]:
# Start counter
counter = 0

# Loop through dataset
for i in tqdm(range(len(restaurants.index))):
    # If 0 in latitude, print street name
    if restaurants.iloc[i, 3:4].values[0] == 0:
        # Print address
        print(restaurants.iloc[i, 0:1].values[0])
        # Add 1 to counter
        counter += 1
    # If large 0
    else:
        # Do nothing
        continue

# Spacer
print() 

# Show counter
print("We are missing coordinates for {counter} restaurants.".format(counter=counter))

# Calculate share of dataset
print("That is {:.2f}% of our dataset.".format(counter/121*100))

100%|██████████| 121/121 [00:00<00:00, 4332.92it/s]Inder Suerst 3
Bra Wo-Allee 1
Otto-Lillienthal-Strasse 19
Speicherstrasse 1
Hanauer Landstrasse 110
Bleichenbruecke 9
Bruesseler Strasse 14
Ander Untertrave 111
Anden Reeperbahnen 2
Enzian Hoefe
Duesseldorferstrasse 162
Muehlstrasse 17
Am Hafendeck 6 - 8
Ander Vorburg 1

We are missing coordinates for 14 restaurants.
That is 11.57% of our dataset.



We can observe that 14 restaurants have no coordinates. A closer look at the respective addresses shows us that some of them are written wrongly (e.g. "Ander Vorburg 1" should be "An der Vorburg 1"). We could try to fix these addresses and run our code section again, however, as the amount restaurants with missing coordinates is below 20% of our dataset, we can accept the discrepancies and exclude these restaurants from our analysis instead:

In [52]:
# Get indexed with lat == 0
index_names = restaurants[(restaurants['LATITUDE'] == 0 )].index

# Kick out entries with lat == 0
restaurants.drop(index_names, inplace=True) 

# Reset index
restaurants.reset_index(drop=True, inplace=True)

# Visualize
print(restaurants.shape)
restaurants.head()

(107, 6)


,STREET,ZIP_CODE,CITY,LATITUDE,LONGITUDE,MUNICIPALITY
0,Gut-Daemme-Strasse 1,52070,Aachen,50.797335,6.107450,None
1,Franziskanergasse 5,92224,Amberg,49.446234,11.855468,None
2,Pearl-S.-Buck-Strasse 12,86156,Augsburg,48.371837,10.865182,None
3,Albert-Schenavsky-Str. 2,86165,Augsburg,48.381857,10.931547,None
4,Maximilianstrasse 83,95444,Bayreuth,49.944196,11.571159,None


Great! Now, we have a data set that we can continue working with. In our next step, we will check which coordinates are lying in which German municipalty. Why are we doing this? Since we do not know the address information for our Facebook data, we need to filter it somehow. Once we know which municipalties are relevant for our L'Osteria restaurants, we can reproduce the steps to assign our Facebook data accordingly.
<br>
<br>
To achieve this, we will use the [turfpy](https://pypi.org/project/turfpy/) library by Omkar Mestry and Sachrin Kharude. The library checks whether a gemoetric point lies within a specified polygon and returns a boolean value. 
<br>
<br>
As the library works with geometric object types, we first need to convert our municipality coordinates to the required object types:

In [94]:
%%time

# Import working libraries
from geojson import Point, Polygon, Feature

# Create temporary variable
copy = copy = municipalities.copy()

# Convert coordinates to polygon objects
copy['coords'] = copy.apply(lambda row: Polygon([row['coords']]), axis=1)

Wall time: 4.2 s


We then can use turfpy to match our L'Osteria coordinates against the individual German border polygons. When a match is found, the respective municipality is stored in a new column in our restaurans variable:

In [95]:
%%time

# Import working libraries
from turfpy.measurement import boolean_point_in_polygon

# Define function to find matching municipality polygon
def find_municipality(lat, lng):
    # Define point 
    point = Feature(geometry=Point((lat, lng)))
    # Check against municipalities
    for i in range(len(copy.index)):
        # Get municipality
        municipality = copy.iloc[i, 0:1].values[0]
        # Get polygon
        polygon = copy.iloc[i, 2:3].values[0]
        # Check if point is in polygon
        if boolean_point_in_polygon(point, polygon) == True:
            # Store municipality in variable
            return municipality
            # Return to main loop
            break

# Execute function
restaurants['MUNICIPALITY'] = restaurants.apply(lambda row: find_municipality(lat=row['LATITUDE'], lng=row['LONGITUDE']), axis=1)

# Visualize
restaurants.head()

Wall time: 2min 28s


,STREET,ZIP_CODE,CITY,LATITUDE,LONGITUDE,MUNICIPALITY
0,Gut-Daemme-Strasse 1,52070,Aachen,50.797335,6.107450,Aachen
1,Franziskanergasse 5,92224,Amberg,49.446234,11.855468,Amberg
2,Pearl-S.-Buck-Strasse 12,86156,Augsburg,48.371837,10.865182,Augsburg
3,Albert-Schenavsky-Str. 2,86165,Augsburg,48.381857,10.931547,Augsburg
4,Maximilianstrasse 83,95444,Bayreuth,49.944196,11.571159,Bayreuth


As you can see, this process takes quite some time. We will therefore save the dataset in a CSV file so that we can upload it, if necessary.


In [70]:
# Save to CSV
restaurants.to_csv(r'C:\Users\maurice.buettgenbach\OneDrive - Aquila Capital Management GmbH\Desktop\Desktop\Private\IBM\10_Capstone project\IBM_The_battle_of_neighborhoods\restaurants_data.csv', index = False)

Let us briefly check whether that worked:

In [71]:
# Load CSV
restaurants = pd.read_csv('restaurants_data.csv')

# Visualize
restaurants.head()

,STREET,ZIP_CODE,CITY,LATITUDE,LONGITUDE,MUNICIPALITY
0,Gut-Daemme-Strasse 1,52070,Aachen,50.797335,6.107450,Aachen
1,Franziskanergasse 5,92224,Amberg,49.446234,11.855468,Amberg
2,Pearl-S.-Buck-Strasse 12,86156,Augsburg,48.371837,10.865182,Augsburg
3,Albert-Schenavsky-Str. 2,86165,Augsburg,48.381857,10.931547,Augsburg
4,Maximilianstrasse 83,95444,Bayreuth,49.944196,11.571159,Bayreuth


Alright - this hopefully will save us some time! In the next step, we have to reproduce the above step and match our facebook data with the German border data. In result, we will be able to filter our Facebook data more conveniently and also add some more geospatial information.
<br>
<br>
However, if you decide to run this code yourself, be aware that this operation takes pretty long as well. It is sinificantly faster than reverse engineering all coordinates in the Facebook dataset but still takes ~1 day to run. So you might as well do somethin else why the computer does its job! The data will also be saved in a CSV file so that we do not need to perform this step everytime we re-run this notebook.

In [73]:
# Create copy from populations variable
copy = population.copy()

# Create temporary variable to store found municipality
found_municipality = []

# Loop through copy to check for municipalties
for i in tqdm(range(len(copy[:3].index))):
    # Define point 
    point = Feature(geometry=Point((copy.iloc[i, 0:1].values[0], copy.iloc[i, 1:2].values[0])))
    # Loop through polygons and check whether point lies in them
    for x in range(len(municipalities.index)):
        # Get municipality
        municipality = municipalities.iloc[x, 0:1].values[0]
        # Get polygon
        polygon = Polygon([municipalities.iloc[x, 2:3].values[0]])
        # Check if point is in polygon
        if boolean_point_in_polygon(point, polygon) == True:
            # Store found municipality in variable
            found_municipality.append(municipality)
            # Return to main loop
            break

# Store municipality in restaurants variable
# population['MUNICIPALITY'] = found_municipality

# # Spacer
# print()
# print()

# # Visualize
# restaurants.head()

100%|██████████| 3/3 [00:55<00:00, 18.41s/it]


In [74]:
found_municipality

['Bad Herrenalb', 'Bad Herrenalb', 'Bad Herrenalb']

## THE LIBRARIES <a name="THE_LIBRARIES" ></a>

In [2]:
# Data wrangling
import pandas as pd
import re

# URL handling
from urllib.request import urlopen
import json

# Web scraping
import requests
from bs4 import BeautifulSoup

# Geo data
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, Feature

# Visualization
from folium import plugins
from folium.plugins import HeatMap
import plotly.express as px
from tqdm import tqdm

# Check out
print('Libraries imported.')

Libraries imported.


In [ ]:
# point = Feature(geometry=Point((-46.6318, -23.5523)))
# polygon = Polygon(
#     [
#         [
#             (-46.653, -23.543),
#             (-46.634, -23.5346),
#             (-46.613, -23.543),
#             (-46.614, -23.559),
#             (-46.631, -23.567),
#             (-46.653, -23.560),
#             (-46.653, -23.543),
#         ]
#     ]
# )
# boolean_point_in_polygon(point, polygon)

In [138]:
# Create temporary variable
df = population.head(1000)

# Create empty list 
addresses = []

# Create counter
counter = 0

# Set geolocator
geolocator = Nominatim(user_agent="german_italian_restaurant")

# Loop through df and get addresses for coordinates
for i in tqdm(range(len(df))):
    # Extract coordinates
    lat = str(df.iloc[i, 0:1].to_numpy()[0])
    lng = str(df.iloc[i, 1:2].to_numpy()[0])
    # Create string from coordinates
    coordinates = str(lat + ", " + lng)
    # Get location with geopy
    location = geolocator.reverse(coordinates)
    # Store in variable
    addresses.append(location.address)

100%|██████████| 1000/1000 [08:19<00:00,  2.00it/s]


In [135]:
# Create address data
address_data = pd.DataFrame()

# Loop through addresses and append them to adress df
for i in tqdm(range(len(addresses))):
    # Split strings
    temp = addresses[i].split(",")
    # Convert to df
    df = pd.DataFrame(temp)
    # Transpose
    df = pd.DataFrame.transpose(df)
    # Concatenate
    frames = [df, address_data]
    address_data = pd.concat(frames)

100%|██████████| 25/25 [00:00<00:00, 464.47it/s]


In [136]:
# Loop through address df and check for inconsistency
for column in tqdm(address_data):
    # Check for NaN values
    if address_data[column].isnull().values.any():
        # If NaN detected, check each row
        for i in range(len(address_data)):
            # Check for NaN in cell
            if pd.isna(address_data.iloc[i, column]) == True:
                # If NaN in cell, shift row
                address_data.iloc[i] = address_data.iloc[i].shift(1)
    else:
        continue

 44%|████▍     | 11/25 [00:00<00:00, 89.18it/s]


In [137]:
print(address_data.shape)
address_data.head()

(25, 11)


,0,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,Bienenzüchterverein Bad Herrenalb,Herrenalb,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,Trampolin oder Pool,Moosbronner Straße,Althof,Bernbach,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,145,Gaistalstraße,Unteres Gaistal,Herrenalb,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,21,Im Wiesengrund,Unteres Gaistal,Herrenalb,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,NaN,28,Wallfahrtstraße,Neusatz,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland


In [ ]:
# Set column names
columns = [
    'STREET_NUMBER',
    'STREET',
    'NEIGHBORHOOD',
    'CITY'
    ]